In [56]:
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import os
import random

In [79]:
driver=None

pagenumber=1

URL=None

def initiate():
    
    global driver
    
    driver=wd.Chrome(executable_path='C:/chromedriver.exe')
    
    driver.maximize_window()
    
    driver.get('https://www.sellerlabs.com/blog/')
    
    sleep(4)
    
    try:
        
        driver.find_element_by_xpath('//*[@id="CybotCookiebotDialogBodyButtonAccept"]').click()
        
    except:
        
        pass

In [58]:
def change_language():
    
    global driver
    
    arrow=driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]/div[4]/div[3]')

    arrow.click()

    swedish=driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[4]/div/div[2]/div[2]/div[2]/div/div[2]/div[93]')

    swedish.click()

In [59]:
def extract_blog():
    
    global driver
    
    try:
    
        text=driver.find_element_by_xpath('//div[@class="p-post__text"]').text
    
    except:
        
        text=None
    
    return text

In [70]:
def google_translate():
    
    global driver
    
#     driver.execute_script('window.open("");')
    
    handles=driver.window_handles
    
    driver.switch_to.window(handles[-1])
    sleep(random.randint(5,10))
    
    driver.get('https://translate.google.co.in/')
    sleep(2)
    
    doc=driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[1]/div[1]/div/div[2]')
    doc.click()
    sleep(1)
    
    inp=driver.find_element_by_xpath('//*[@id="tlid-file-input"]')
    inp.send_keys(os.path.abspath('SellerLabs/Page.txt'))
 
    sleep(2)
    
    change_language()
    
    translate=driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[1]/div[2]/div[1]/div[2]/div/form/div[2]/div[2]/input')
    translate.click()
    
    sleep(3)
    
    translated_text=driver.find_element_by_tag_name('body').text
        
#     driver.close()
    
#     driver.switch_to.window(handles[0])
    
    delete_file()
    
    return translated_text
    

In [71]:
def write_to_file(text):
    
    global URL
    
    global pagenumber
    
    file_name='SellerLabs/Page' + str(pagenumber)+'.txt'
    
    pagenumber+=1
    
    with open(file_name,'a') as file:
        
        file.write('URL:{}'.format(URL))
        
        file.write('\n\n')
        
        for i in text:
            
            try:
                
                file.write(i)
            
            except:
                
                pass
        
        file.write('\n\n\n\n')

In [72]:
def create_temp_file(text):
    
    with open('SellerLabs/Page.txt','w') as file:
        
        for i in text:
        
            try:
                
                file.write(i)
            
            except:
                
                pass
    

In [73]:
def delete_file():
    
    try:
        
        os.remove('SellerLabs/Page.txt')
        
        print('File deleted successfully !')
        
    except:
        
        pass

In [80]:
def next_blog():
    
    global driver
    
    global URL
    
    tags=driver.find_elements_by_xpath('//div[@class="p-blog__post"]')
    
    for tag in tags:
        
        link=tag.find_element_by_tag_name('a').get_attribute('href')
        
        driver.execute_script('window.open("");')
        
        handles=driver.window_handles
        
        driver.switch_to.window(handles[-1])
        
        driver.get(link)
        
        URL=link
        
        text=extract_blog()
        
        if text!=None:

            create_temp_file(text)

            translated=google_translate()

            write_to_file(translated)

            delete_file()
        
        driver.close()

        driver.switch_to.window(handles[0])


In [81]:
def next_page():
    
    global driver
    
    try:
        
        next_=driver.find_element_by_xpath('//*[@id="posts_container"]/div/div/a[3]')
        
        if next_.text=='›':
        
            next_.click()
        
            sleep(2)
            
            return True
        
        else:
            
            return False
        
    except:
        
        return False
        
        pass
    

In [82]:
def create_directory():
    
    print('In dir')
    
    try:
        
        os.mkdir('SellerLabs')
        
    except:
        
        pass

In [83]:
def executor():
    
    create_directory()
    
    initiate()
    
    load_all()
    
    next_blog()
        

In [84]:
def load_all():
    
    global driver
    
    while True:
        
        try:
            
            load=driver.find_elements_by_partial_link_text('Load more posts')
            
            if len(load)<1:
                
                break
                
            else:
                
                for i in load:
                    
                    i.click()
                    
        except: pass
        
    driver.find_element_by_tag_name('body').send_keys(Keys.HOME)

In [86]:
# initiate()
# executor()